## Final Project Submission

Please fill out: 
* Student name:   1.

                  2. Hannah Nyambura 

                  3. 

                  4. 

                  5.
                 
* Student pace:      HYBRID
* Scheduled project review date/time: 
* Instructor name:     SAMUEL KARU
* Blog post URL:


# MICROSOFT MOVIE STUDIO INSIGHT
---


# PROJECT OVERVIEW
- The aim of this project is to analyze which type of films are currently doing the best at the box office to help the company decide which type of films to create.

# BUSINESS PROBLEM
- A company has decided to create a new movie studio, but they don’t know anything about creating movies. We are charged with exploring what types of films are currently doing the best at the box office. We must then translate those findings into actionable insights that the head of this company's new movie studio can use to help decide what type of films to create.



# OBJECTIVES
* To determine which movie genres perform highly in terms of revenue and viewer rating.
* To determine which directors and actors are linked to the highest performing movies.
* To determine whether movie budget affects the overall performance of a movie.

# PROJECT GOAL

- To identify the key factors that drive movie success (financially and with audiences) in order to guide the new movie studio on what types of films to produce, invest in, and promote.

## Importing the necessary libraries

In [3]:
import pandas as pd
import gzip
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats

## Loading Datasets

### 1. im.db Database

In [1]:
# Unzip the archive manually 
import zipfile
import os

zip_path = './zippedData/im.db.zip'  # replace with actual file
extract_path = './zippedData/'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Unzipping done!")


Unzipping done!


In [4]:
# Load the imdb database
conn = sqlite3.connect('./zippedData/im.db')
pd.read_sql("SELECT name FROM sqlite_master WHERE type=='table';" ,conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


**Tables to use:**
>* movie_basics
>* directors
>* movie_ratings
>* persons

In [5]:
# loading movie_ratings
pd.read_sql("SELECT * FROM movie_ratings", conn)

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


In [6]:
# display movie_basics table
pd.read_sql("SELECT * FROM movie_basics", conn).head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [7]:
# display persons table
pd.read_sql("SELECT * FROM persons", conn).head()

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"


In [8]:
# display directors table
pd.read_sql("SELECT * FROM directors", conn).head()

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502


### 2. tn.movie_budgets Dataset

In [9]:
# Load the tn.movie_budgets dataset
movie_budgets = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz', compression='gzip', index_col='id')
movie_budgets.head()

,release_date,movie,production_budget,domestic_gross,worldwide_gross
id,,,,,
1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


# Data Cleaning

### 1. imdb Dataset

In [10]:
#Create a function to check for duplicates in a table
def duplicates(table_name): #defines a function named duplicates that takes one input parameter:
    return pd.read_sql(f"SELECT * FROM {table_name}", conn).duplicated().sum()

`.duplicated()`
This Pandas method returns a Boolean Series that flags whether each row in the DataFrame is a duplicate of a previous row (based on all columns by default).

Count of Duplicates and Non-Duplicates
`.value_counts()`
This counts how many True (duplicate rows) and False (unique rows) values are returned by `.duplicated()`.

In [11]:
# Create a function to check for missing values in a table
def missing(table_name):
    return pd.read_sql(f"SELECT * FROM {table_name}", conn).isna().sum()

`pd.read_sql(...)`: Reads all rows from the specified SQL table into a Pandas DataFrame.

`.isna()`: Checks each value in the DataFrame for missing data (i.e., NaN).

`.sum()`: Aggregates the number of missing values in each column.

In [12]:
#Check for duplicates
duplicates('movie_basics')

0

This means all 146,144 rows are unique, and no duplicates were found in the `movie_basics table`, indicating good data integrity for this dataset.

In [13]:
# Check for missing values
missing('movie_basics')

movie_id               0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
dtype: int64

Ther are 21 missing values in `original_title`, 31739 in runtime_minutes and 5408 in `genres`.

WE fill `runtime_minutes` with median for these reason:
> The median is a robust statistic that better represents the "typical" movie length when the data is skewed.

> It prevents distortion in the dataset that could happen if extremely long or short runtimes heavily influence the mean.

In [14]:
# Assign the movie_basics table to a DataFrame
movie_basics = pd.read_sql("SELECT * FROM movie_basics", conn)

# Fillthe missing values in the runtime_minutes column with the median value of the column
movie_basics['runtime_minutes'] = movie_basics['runtime_minutes'].fillna(movie_basics['runtime_minutes'].median())
movie_basics.isna().sum()

movie_id              0
primary_title         0
original_title       21
start_year            0
runtime_minutes       0
genres             5408
dtype: int64

> The number of remaining missing rows was likely small relative to the overall dataset, so removing them minimized data loss.



In [15]:
# Drop rows with missing values
movie_basics.dropna(inplace=True)
movie_basics.isna().sum()

movie_id           0
primary_title      0
original_title     0
start_year         0
runtime_minutes    0
genres             0
dtype: int64

In [16]:
# Save the cleaned DataFrame back to the database
movie_basics.to_sql('movie_basics',conn, if_exists='replace', index=False)

In [17]:
# Check for duplicates
duplicates('movie_ratings')

0

In [18]:
# Check for missing values
missing('movie_ratings')

movie_id         0
averagerating    0
numvotes         0
dtype: int64

There are no duplicates and missing values in `movie_rating`.
Thats make the table perfect to work with.

In [19]:
# Check for duplicates
duplicates('directors')

127639

> The duplicate check showed that 127,639 rows were exact duplicates, indicating a high level of redundancy in the directors table.

> Using drop_duplicates() removes all repeated rows, keeping only the first occurrence of each.

This ensures that:

> The dataset is clean and efficient.

> Analyses involving director data (e.g., frequency counts, joins with other tables) are not skewed or inflated by repeated entries.


In [20]:
# Assign the directors table to a DataFrame
directors = pd.read_sql('SELECT * FROM directors',conn)

# Drop duplicates in the directors DataFrame
directors.drop_duplicates(inplace=True)
directors.duplicated().sum()

0

In [21]:
# Save the cleaned DataFrame back to the database
directors.to_sql('directors',conn, if_exists='replace',index=False)

> The step above is important for: 
- It ensures the cleaned data is stored persistently in the database for use in further analysis or merging with other tables.

- Replacing the old table avoids confusion or errors caused by outdated, duplicate-filled data.

In [22]:
# Check for missing values
missing('directors')

movie_id     0
person_id    0
dtype: int64

### 2. Movie_Budgets Dataset

In [25]:
movie_budgets.head()

,release_date,movie,production_budget,domestic_gross,worldwide_gross
id,,,,,
1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [26]:
# Drop the columns that are not needed for the analysis
movie_budgets = movie_budgets.drop(columns = ['release_date','domestic_gross'])


In [27]:
# Remove non-numeric characters and convert to numeric
for i in ['$',',']:
    movie_budgets['production_budget'] = movie_budgets['production_budget'].str.replace(i, '')
    movie_budgets['worldwide_gross'] = movie_budgets['worldwide_gross'].str.replace(i, '')
movie_budgets['production_budget'] = pd.to_numeric(movie_budgets['production_budget'])
movie_budgets['worldwide_gross'] = pd.to_numeric(movie_budgets['worldwide_gross'])
movie_budgets.head()

,movie,production_budget,worldwide_gross
id,,,
1,Avatar,425000000,2776345279
2,Pirates of the Caribbean: On Stranger Tides,410600000,1045663875
3,Dark Phoenix,350000000,149762350
4,Avengers: Age of Ultron,330600000,1403013963
5,Star Wars Ep. VIII: The Last Jedi,317000000,1316721747


In [28]:
# Check summary information for the df
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5782 entries, 1 to 82
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   movie              5782 non-null   object
 1   production_budget  5782 non-null   int64 
 2   worldwide_gross    5782 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 180.7+ KB


In [29]:
# Check for duplicates in the movie_budgets DataFrame
movie_budgets.duplicated().value_counts()

False    5782
dtype: int64

> the `movie_budgets dataset` is clean.

In [30]:
movie_budgets.isna().sum()

movie                0
production_budget    0
worldwide_gross      0
dtype: int64

In [ ]:
# data cleaning made easier